In [1]:
import os
import sys
import yaml
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F
import time
from torch.utils.data import DataLoader

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%pip install wandb -qU
%matplotlib inline

# Get the current working directory
notebook_dir = notebook_dir = os.path.dirname(os.path.abspath("__file__"))  
project_dir = os.path.abspath(os.path.join(notebook_dir, '..')) 
if project_dir not in sys.path:
    sys.path.append(project_dir)

from src import (get_transforms, load_data, split_data, set_seeds, 
                 verify_splits, verify_data, plot_species_grid,
                 verify_loader_transforms)
from src.data_utils import ImagesDataset
from src.models import build_resnet50_basic
from src.train import setup_training, evaluate, train

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Locate the YAML file relative to the notebook's location
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# You need to update this path to your new .yaml file
config_path = os.path.join(notebook_dir, "../configs/default_cuda.yaml")

# Load the YAML file
with open(config_path, "r") as f:
    config = yaml.safe_load(f)
print(config)

{'device': 'cuda', 'model': {'num_classes': 8, 'architecture': 'efficientnet', 'freeze_backbone': False, 'dropout': 0.2, 'hidden_units1': 100}, 'train': {'batch_size': 4, 'epochs': 2, 'lr': 0.001, 'momentum': 0.9, 'optimizer': 'sgd', 'criterion': 'cross_entropy'}, 'experiment': {'seed': 42, 'experiment_name': 'efficientnet'}, 'transforms': {'resize': [480, 480], 'horizontal_flip': 0.5, 'rotate': 15, 'jitter': {'brightness': 0.2, 'contrast': 0.2, 'saturation': 0.2, 'hue': 0.1}, 'custom': {'block_timestamp': True}}, 'log': {'img_count': 50}}


In [3]:
print(torch.__version__)
print(torch.backends.mps.is_available())
device = config["device"]
print(f"Running on device: {device}")

2.5.1
False
Running on device: cuda


In [5]:
train_features, test_features, train_labels, species_labels = load_data()
species_labels

['antelope_duiker',
 'bird',
 'blank',
 'civet_genet',
 'hog',
 'leopard',
 'monkey_prosimian',
 'rodent']

In [10]:

train_labels_antelope_duiker = train_labels[["antelope_duiker", "bird"]]

train_labels_bird = train_labels[["bird", "blank"]]

train_labels_blank = train_labels[["blank", "bird"]]

train_labels_civet_genet = train_labels[["civet_genet", "blank"]]

train_labels_hog = train_labels[["hog", "blank"]]

train_labels_leopard = train_labels[["leopard", "hog"]]

train_labels_monkey_prosimian = train_labels[["monkey_prosimian", "blank"]]

train_labels_rodent = train_labels[["rodent", "blank"]]



In [ ]:
# Get transforms
train_transforms, val_transforms = get_transforms(config)

In [11]:
set_seeds(config["experiment"]["seed"])

X_train_antelope_duiker, X_val_antelope_duiker, y_train_antelope_duiker, y_val_antelope_duiker = split_data(train_features, train_labels_antelope_duiker, type='sites')

X_train_bird, X_val_bird, y_train_bird, y_val_bird = split_data(train_features, train_labels_bird, type='sites')

X_train_blank, X_val_blank, y_train_blank, y_val_blank = split_data(train_features, train_labels_blank, type='sites')

X_train_civet_genet, X_val_civet_genet, y_train_civet_genet, y_val_civet_genet = split_data(train_features, train_labels_civet_genet, type='sites')

X_train_hog, X_val_hog, y_train_hog, y_val_hog = split_data(train_features, train_labels_hog, type='sites')

X_train_leopard, X_val_leopard, y_train_leopard, y_val_leopard = split_data(train_features, train_labels_leopard, type='sites')

X_train_monkey_prosimian, X_val_monkey_prosimian, y_train_monkey_prosimian, y_val_monkey_prosimian = split_data(train_features, train_labels_monkey_prosimian, type='sites')

X_train_rodent, X_val_rodent, y_train_rodent, y_val_rodent = split_data(train_features, train_labels_rodent, type='sites')